In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba

In [2]:
class MSDSampler:
    def __init__(self, maxCorrelationTime, maxOrigins, numberOfParticles):
        self.maxCorrelationTime = maxCorrelationTime
        self.maxOrigins = maxOrigins
        self.numberOfParticles = numberOfParticles
        
        self.msd = np.zeros((maxCorrelationTime, 2))
        self.counts = np.zeros(maxCorrelationTime, dtype=np.int32)
        self.originPositions = np.zeros((maxOrigins, numberOfParticles, 2))
        self.originTimes = np.zeros(maxOrigins, dtype=np.int32)
 
    def sample(self, position, cycle):
        self.originTimes[cycle % self.maxOrigins] = cycle
        self.originPositions[cycle % self.maxOrigins] = position
        
        for i in range(min(cycle, self.maxOrigins)):
            time_difference = cycle - self.originTimes[i]
            if time_difference < self.maxCorrelationTime:
                self.counts[time_difference] += 1
                self.msd[time_difference] += np.mean((position - self.originPositions[i])**2, axis=0)
                
    def getMSD(self):
        return self.msd / self.counts[:, None]


In [6]:
latticeVectors = np.array([[1,0], [0,1], [-1,0], [0,-1]], dtype=np.int32)

@numba.njit
def randomWalk2D(numberOfCycles: int, numberOfParticles: int, latticeSize: int):
    lattice = np.zeros((latticeSize, latticeSize), dtype=np.int32)
    
    # select random lattice sites to initialize particles
    indices = np.random.choice(latticeSize**2, size=numberOfParticles, replace=False)
    xPositions = indices % latticeSize
    yPositions = indices // latticeSize
    
    positions = np.column_stack((xPositions, yPositions))
    unwrappedPositions = positions.copy()
    
    msdSampler = MSDSampler(500, 50, numberOfParticles)
    
    for cycle in range(numberOfCycles):
        # Select particle and displacement
        particleIndex = np.random.choice(numberOfParticles)
        dx = latticeVectors[np.random.choice(4)]
        xold, yold = positions[particleIndex]
        
        # Get new position and wrap in box
        newPosition = positions[particleIndex] + dx
        newPosition %= latticeSize
        xnew, ynew = newPosition
        
        # Check if lattice site is occupied
        if lattice[xold, yold] != 1:
            
            lattice[xold, yold] = 0
            lattice[xnew, ynew] = 1
            positions[particleIndex] = newPosition
            unwrappedPositions[particleIndex] += dx
    
        msdSampler.sample(unwrappedPositions, cycle)
    return msdSampler.getMSD()

In [7]:
numberOfParticles = 100
latticeSize = 20
numberOfCycles = 100000

In [8]:
msd = randomWalk2D(numberOfCycles, numberOfParticles, latticeSize)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'MSDSampler':[0m [1m[1mCannot determine Numba type of <class 'type'>[0m
[1m
File "../../../../../../var/folders/wl/5zqkqrjx4mnb22f8316bnkf00000gn/T/ipykernel_21000/1702450126.py", line 15:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m

In [ ]:
fig, ax = plt.subplots()
ax.plot(msd[:, 0])
ax.plot(msd[:, 1])